In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
    

In [2]:
!pip install transformers

In [28]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=0b2e0679cdb5470c9a7971a87eef9695e90e78bf74af069be6d1427513cb5d8c
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
import datasets
import transformers
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

conll2003 Dataset for NER

In [4]:
dataset= datasets.load_dataset("conll2003", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [6]:
dataset['train'].features['pos_tags']

Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None)

In [7]:
dataset['train'].features['chunk_tags']

Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None)

In [8]:
dataset['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [9]:
dataset['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [10]:
dataset.shape

{'train': (14041, 5), 'validation': (3250, 5), 'test': (3453, 5)}

Getting BERT Tokenizer

In [11]:
tokenizer= BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
random_example= dataset['train'][100]
random_example

{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0]}

In [13]:
tokenized_input= tokenizer(random_example['tokens'], is_split_into_words= True)
tokenized_input

{'input_ids': [101, 10958, 21891, 12933, 2003, 12788, 2039, 2010, 2744, 2004, 6059, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
tokens= tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

tokens

['[CLS]',
 'ra',
 '##bino',
 '##vich',
 'is',
 'winding',
 'up',
 'his',
 'term',
 'as',
 'ambassador',
 '.',
 '[SEP]']

In [15]:
word_ids= tokenized_input.word_ids()
word_ids

[None, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]

In [16]:
len(random_example['ner_tags'])

9

In [17]:
len(tokenized_input['input_ids'])

13

#Problem

the input ids returned by tokenizer (13) is larger than the list of labels that our dataset contain (9)

reasons (the tokenizer added following changes):
- special tokens at start CLS, and at end SEP
- subword tokens Rabinovich= 'ra', '##bino', '##vich'

#Solution

- add -100 for special tokens (for CLS and for SEP)
- mask the subwords

#why -100?

cuz during training torch will ignore -100 index (In PyTorch the cross-entropy loss class torch.nn.CrossEntropyLoss has an attribute called ignore_index whose value is –100. This index is ignored during training)

In [18]:
def tokenize_and_adjust_labels(dataset, label_all_tokens=True):
    # Tokenize the input tokens from the dataset
    tokenized_inputs = tokenizer(dataset["tokens"], truncation=True, is_split_into_words=True)

    # Initialize an empty list to store the adjusted labels
    labels = []

    # Iterate over the NER tags in the dataset
    for i, label in enumerate(dataset["ner_tags"]):
        # Get the word IDs for the current sentence in the batch
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        # Initialize the previous word index to None
        previous_word_index = None
        # Initialize an empty list to store the labels for the current sentence
        label_ids = []

        # Iterate over the word indices in the tokenized input
        for word_index in word_ids:
            # If the word index is None (e.g., special tokens like [CLS] and [SEP])
            if word_index is None:
                label_ids.append(-100)  # Assign -100 to these tokens

            # If the current word index is different from the previous one
            elif word_index != previous_word_index:
                label_ids.append(label[word_index])  # Assign the corresponding label

            # If the current word index is the same as the previous one (subword)
            else:
                label_ids.append(label[word_index] if label_all_tokens else -100)  # Assign label or -100 based on the flag

            # Update the previous word index
            previous_word_index = word_index

        # Append the labels for the current sentence to the labels list
        labels.append(label_ids)

    # Add the labels to the tokenized inputs
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [19]:
test = tokenize_and_adjust_labels({'tokens': [dataset['train'][50]['tokens']], 'ner_tags': [dataset['train'][50]['ner_tags']]})

In [20]:
test

{'input_ids': [[101, 6728, 2884, 12943, 2362, 2007, 2236, 9693, 2234, 1999, 2117, 2173, 2007, 4749, 1010, 25717, 8819, 2015, 1010, 2385, 1012, 1018, 3867, 1997, 1996, 3452, 3275, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 3, 4, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}

In [21]:
tokenized_dataset= dataset.map(
    tokenize_and_adjust_labels,
    batched=True
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [22]:
tokenized_dataset['train'][100]

{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  10958,
  21891,
  12933,
  2003,
  12788,
  2039,
  2010,
  2744,
  2004,
  6059,
  1012,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -100]}

Adding BERT "uncased" model

In [23]:
model= AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=9)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Specifying Training Arguments

In [49]:
from transformers import TrainingArguments

args= TrainingArguments(
    "ner-model",
    evaluation_strategy= "epoch",
    learning_rate= 2e-5,
    per_device_train_batch_size= 16,
    per_device_eval_batch_size= 16,
    num_train_epochs= 3,
    weight_decay= 0.01

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


#data_collator

data_collator handles padding and prepares the input sequences and labels for training. The resulting batch is a dictionary with keys like input_ids, attention_mask, and labels, each containing padded and properly formatted tensors ready for model training.

In [50]:
data_collator= DataCollatorForTokenClassification(tokenizer)

In [51]:
data_collator

DataCollatorForTokenClassification(tokenizer=BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100, 

# seqeval
seqeval is a Python framework for sequence labeling evaluation.
seqeval can evaluate the performance of chunking tasks such as named-entity recognition, part-of-speech tagging, semantic role labeling and so on.

In [52]:
metric= datasets.load_metric('seqeval', trust_remote_code=True)

In [53]:
metric

Metric(name: "seqeval", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of shape (n_samples,), weights for indi

In [54]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [55]:
label_list = dataset["train"].features["ner_tags"].feature.names

In [56]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [57]:
import numpy as np

def compute_metrics(eval_pred):
    pred_logits, labels = eval_pred

    # Get the predictions by taking the argmax of the logits
    pred_logits = np.argmax(pred_logits, axis=2)

    # Initialize lists to store the predictions and true labels
    predictions = []
    true_labels = []

    # Iterate over the predictions and true labels
    for prediction, label in zip(pred_logits, labels):
        # Convert the indices to label names and filter out -100
        pred_labels = [label_list[p] for p, l in zip(prediction, label) if l != -100]
        true_labels_instance = [label_list[l] for p, l in zip(prediction, label) if l != -100]

        # Append the results to the lists
        predictions.append(pred_labels)
        true_labels.append(true_labels_instance)

    # Compute the evaluation metrics
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
        'precision': results['overall_precision'],
        'recall': results['overall_recall'],
        'f1': results['overall_f1'],
        'accuracy': results['overall_accuracy']
    }


In [58]:
from transformers import Trainer

trainer= Trainer(
    model,
    args,
    train_dataset= tokenized_dataset['train'],
    eval_dataset= tokenized_dataset['validation'],
    data_collator= data_collator,
    tokenizer= tokenizer,
    compute_metrics= compute_metrics
)


In [59]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.005600,0.080214,0.940986,0.941828,0.941407,0.985813
2,0.005500,0.076451,0.942150,0.949211,0.945667,0.986322
3,0.002900,0.076696,0.941379,0.948540,0.944946,0.986449


TrainOutput(global_step=2634, training_loss=0.004120232360600158, metrics={'train_runtime': 653.5041, 'train_samples_per_second': 64.457, 'train_steps_per_second': 4.031, 'total_flos': 1020143109346326.0, 'train_loss': 0.004120232360600158, 'epoch': 3.0})

In [60]:
model.save_pretrained('./ner-bert-model')

In [61]:
tokenizer.save_pretrained('./ner-bert-tokenizer')

('./ner-bert-tokenizer/tokenizer_config.json',
 './ner-bert-tokenizer/special_tokens_map.json',
 './ner-bert-tokenizer/vocab.txt',
 './ner-bert-tokenizer/added_tokens.json',
 './ner-bert-tokenizer/tokenizer.json')

In [62]:
from transformers import pipeline

model= AutoModelForTokenClassification.from_pretrained('./ner-bert-model')
tokenizer= BertTokenizerFast.from_pretrained('./ner-bert-tokenizer')

ner_pipeline= pipeline('ner', model= model, tokenizer= tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [63]:
test_example= 'John lives in New York and works for the United Nations.'
ner_results= ner_pipeline(test_example)

print(ner_results)

[{'entity': 'LABEL_1', 'score': 0.9985078, 'index': 1, 'word': 'john', 'start': 0, 'end': 4}, {'entity': 'LABEL_0', 'score': 0.99971575, 'index': 2, 'word': 'lives', 'start': 5, 'end': 10}, {'entity': 'LABEL_0', 'score': 0.99954563, 'index': 3, 'word': 'in', 'start': 11, 'end': 13}, {'entity': 'LABEL_5', 'score': 0.99972135, 'index': 4, 'word': 'new', 'start': 14, 'end': 17}, {'entity': 'LABEL_6', 'score': 0.9996569, 'index': 5, 'word': 'york', 'start': 18, 'end': 22}, {'entity': 'LABEL_0', 'score': 0.99994206, 'index': 6, 'word': 'and', 'start': 23, 'end': 26}, {'entity': 'LABEL_0', 'score': 0.99991286, 'index': 7, 'word': 'works', 'start': 27, 'end': 32}, {'entity': 'LABEL_0', 'score': 0.99976796, 'index': 8, 'word': 'for', 'start': 33, 'end': 36}, {'entity': 'LABEL_0', 'score': 0.99966156, 'index': 9, 'word': 'the', 'start': 37, 'end': 40}, {'entity': 'LABEL_3', 'score': 0.99972445, 'index': 10, 'word': 'united', 'start': 41, 'end': 47}, {'entity': 'LABEL_4', 'score': 0.9994531, 'in

In [64]:
print(label_list)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
